In [1]:
import sys,os,json
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.colors import ListedColormap
import seaborn as sns
import networkx as nx
# from textblob import TextBlob
# from polyglot.detect import Detector
import operator
# import PlotUtil
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# import foursq_helper as foursq
# from yelp_helper import *

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##########################################
########## data set declaration ##########
path_processed = "data/processed/"
path_final = "data/final/"

##################################
########## End of Setup ##########

##### Geographical Change #####
import geopy
from geopy import distance as geopy_distance
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)
print (geopy.distance.distance(coords_1, coords_2).km)

279.35290160430094


In [ ]:
np.save("data/opp_X_train.npy", X_train)
np.save("data/opp_X_test.npy", X_test)
np.save("data/opp_y_train.npy", y_train)
np.save("data/opp_y_test.npy", y_test)

In [2]:
X_train = np.load("data/opp_X_train.npy")
X_test = np.load("data/opp_X_test.npy")
y_train = np.load("data/opp_y_train.npy")
y_test = np.load("data/opp_y_test.npy")

In [ ]:
np.save("data/opp_X_train_D1.npy", X_train_D1)

In [39]:
X_train.shape

(14147, 24, 113)

## TensorFlow Implementation !

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from sklearn.metrics import *
from sklearn.metrics import fbeta_score
from keras import backend as K

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

Using TensorFlow backend.


In [37]:
temp = LabelEncoder().fit_transform(y_test)

In [40]:
temp.shape

(9894,)

In [4]:
Counter(y_train)

Counter({0: 32348,
         1: 864,
         2: 887,
         3: 806,
         4: 846,
         5: 921,
         6: 850,
         7: 666,
         8: 628,
         9: 490,
         10: 413,
         11: 457,
         12: 416,
         13: 566,
         14: 564,
         15: 904,
         16: 3246,
         17: 623})

In [5]:
X_train = X_train[y_train[:]>0]
y_train = y_train[y_train[:]>0]
y_train = y_train[:] - 1

X_test = X_test[y_test[:]>0]
y_test = y_test[y_test[:]>0]
y_test = y_test[:] - 1

In [6]:
Counter(y_train)

Counter({0: 864,
         1: 887,
         2: 806,
         3: 846,
         4: 921,
         5: 850,
         6: 666,
         7: 628,
         8: 490,
         9: 413,
         10: 457,
         11: 416,
         12: 566,
         13: 564,
         14: 904,
         15: 3246,
         16: 623})

In [7]:
X_train.shape

(14147, 24, 113)

In [26]:
def makeDataD2OneClass(class_num_D2, X_train, y_train, X_test, y_test):

    X_train_D1 = X_train[y_train[:] != class_num_D2]
    X_train_D2 = X_train[y_train[:] == class_num_D2]

    y_train_D1 = y_train[y_train[:] != class_num_D2]
    y_train_D2 = y_train[y_train[:] == class_num_D2]

    X_test_D1 = X_test[y_test[:] != class_num_D2]
    X_test_D2 = X_test[y_test[:] == class_num_D2]

    y_test_D1 = y_test[y_test[:] != class_num_D2]
    y_test_D2 = y_test[y_test[:] == class_num_D2]
    
    return X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2

# [True if s in class_l else False for s in y_train]
def makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test):

    X_train_D1 = X_train[[True if x in class_dict_D1 else False for x in y_train]]
    X_train_D2 = X_train[[True if x in class_dict_D2 else False for x in y_train]]

    y_train_D1 = y_train[[True if x in class_dict_D1 else False for x in y_train]]
    y_train_D2 = y_train[[True if x in class_dict_D2 else False for x in y_train]]

    X_test_D1 = X_test[[True if x in class_dict_D1 else False for x in y_test]]
    X_test_D2 = X_test[[True if x in class_dict_D2 else False for x in y_test]]

    y_test_D1 = y_test[[True if x in class_dict_D1 else False for x in y_test]]
    y_test_D2 = y_test[[True if x in class_dict_D2 else False for x in y_test]]
    
    return X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2

def makeClassDictD1(class_dict_D2, num_classes):
    class_list = []
    for i in range(num_classes):
        if i not in class_dict_D2:
            class_list.append(i)
    return set(class_list)

In [27]:
def vanilaLSTMs():
    model = Sequential()
    model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
    return model

In [28]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps = 24
num_stack = 1
hidden_layer_size = 128

In [29]:
class_dict_D2 = {0,1,2,3,4,5,6,7,8}
class_dict_D1 = makeClassDictD1(class_dict_D2, num_classes)
print("==== Adding Half More Classes ====")
print("class dict D1: ", class_dict_D1)
print("class dict D2: ", class_dict_D2)
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))


==== Adding Half More Classes ====
class dict D1:  {9, 10, 11, 12, 13, 14, 15, 16}
class dict D2:  {0, 1, 2, 3, 4, 5, 6, 7, 8}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 7189 samples, validate on 757 samples
Epoch 1/30
7189/7189 [==============================] - 9s 1ms/step - loss: 1.2317 - sparse_categorical_accuracy: 0.5574 - val_loss: 1.1894 - val_sparse_categorical_accuracy: 0.5139
Epoch 2/30
7189/7189 [==============================] - 8s 1ms/step - loss: 0.8011 - sparse_categorical_accuracy: 0.6820 - val_loss: 1.3218 - val_sparse_categorical_accuracy: 0.5601
Epoch 3/30
7189/7189 [==============================] - 9s 1ms/step - loss: 0.6974 - sparse_categorical_accuracy: 0.7200 - val_loss: 0.8464 - val_sparse_categorical_accuracy: 0.6988
Epoch 4/30
7189/7189 [=======================

Epoch 8/9
6958/6958 [==============================] - 7s 960us/step - loss: 0.7983 - sparse_categorical_accuracy: 0.6523 - val_loss: 4.6468 - val_sparse_categorical_accuracy: 0.2511
Epoch 9/9
6958/6958 [==============================] - 7s 997us/step - loss: 0.7805 - sparse_categorical_accuracy: 0.6625 - val_loss: 4.5279 - val_sparse_categorical_accuracy: 0.3380
Performances after Epoch: 10
acc_D1: 0.000000
acc_D2: 0.622222
Train on 6958 samples, validate on 1657 samples
Epoch 1/20
6958/6958 [==============================] - 7s 955us/step - loss: 0.7374 - sparse_categorical_accuracy: 0.6845 - val_loss: 4.4421 - val_sparse_categorical_accuracy: 0.2637
Epoch 2/20
6958/6958 [==============================] - 7s 1ms/step - loss: 0.7588 - sparse_categorical_accuracy: 0.6786 - val_loss: 4.2811 - val_sparse_categorical_accuracy: 0.3102
Epoch 3/20
6958/6958 [==============================] - 6s 879us/step - loss: 0.7119 - sparse_categorical_accuracy: 0.6919 - val_loss: 4.3665 - val_sparse_ca

In [30]:
class_dict_D2 = {9,10,11,12,13,14,15,16}
class_dict_D1 = makeClassDictD1(class_dict_D2, num_classes)
print("==== Adding Half More Classes ====")
print("class dict D1: ", class_dict_D1)
print("class dict D2: ", class_dict_D2)
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))


==== Adding Half More Classes ====
class dict D1:  {0, 1, 2, 3, 4, 5, 6, 7, 8}
class dict D2:  {9, 10, 11, 12, 13, 14, 15, 16}
Train on 6958 samples, validate on 900 samples
Epoch 1/30
6958/6958 [==============================] - 8s 1ms/step - loss: 1.7994 - sparse_categorical_accuracy: 0.2639 - val_loss: 1.5429 - val_sparse_categorical_accuracy: 0.2544
Epoch 2/30
6958/6958 [==============================] - 6s 903us/step - loss: 1.3379 - sparse_categorical_accuracy: 0.3977 - val_loss: 1.6019 - val_sparse_categorical_accuracy: 0.3800
Epoch 3/30
6958/6958 [==============================] - 7s 975us/step - loss: 1.2219 - sparse_categorical_accuracy: 0.4363 - val_loss: 1.2602 - val_sparse_categorical_accuracy: 0.4611
Epoch 4/30
6958/6958 [==============================] - 6s 912us/step - loss: 1.1671 - sparse_categorical_accuracy: 0.4665 - val_loss: 1.1850 - val_sparse_categorical_accuracy: 0.5067
Epoch 5/30
6958/6958 [==============================] - 6s 876us/step - loss: 1.0807 - spars

7189/7189 [==============================] - 6s 768us/step - loss: 0.4114 - sparse_categorical_accuracy: 0.8264 - val_loss: 4.6440 - val_sparse_categorical_accuracy: 0.3518
Epoch 4/20
7189/7189 [==============================] - 5s 764us/step - loss: 0.3643 - sparse_categorical_accuracy: 0.8487 - val_loss: 4.7212 - val_sparse_categorical_accuracy: 0.3331
Epoch 5/20
7189/7189 [==============================] - 6s 770us/step - loss: 0.3667 - sparse_categorical_accuracy: 0.8457 - val_loss: 4.4708 - val_sparse_categorical_accuracy: 0.3585
Epoch 6/20
7189/7189 [==============================] - 5s 761us/step - loss: 0.3852 - sparse_categorical_accuracy: 0.8360 - val_loss: 4.6103 - val_sparse_categorical_accuracy: 0.3530
Epoch 7/20
7189/7189 [==============================] - 5s 762us/step - loss: 0.4077 - sparse_categorical_accuracy: 0.8221 - val_loss: 4.0302 - val_sparse_categorical_accuracy: 0.3150
Epoch 8/20
7189/7189 [==============================] - 6s 807us/step - loss: 0.3551 - spar

In [31]:
class_dict_D2 = {0,2,4,6,8,10,12,14,16}
class_dict_D1 = makeClassDictD1(class_dict_D2, num_classes)
print("==== Adding Half More Classes ====")
print("class dict D1: ", class_dict_D1)
print("class dict D2: ", class_dict_D2)
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))


==== Adding Half More Classes ====
class dict D1:  {1, 3, 5, 7, 9, 11, 13, 15}
class dict D2:  {0, 2, 4, 6, 8, 10, 12, 14, 16}
Train on 7850 samples, validate on 861 samples
Epoch 1/30
7850/7850 [==============================] - 7s 916us/step - loss: 1.1915 - sparse_categorical_accuracy: 0.5559 - val_loss: 1.2393 - val_sparse_categorical_accuracy: 0.5668
Epoch 2/30
7850/7850 [==============================] - 6s 763us/step - loss: 0.7629 - sparse_categorical_accuracy: 0.6780 - val_loss: 1.0707 - val_sparse_categorical_accuracy: 0.5691
Epoch 3/30
7850/7850 [==============================] - 6s 742us/step - loss: 0.6801 - sparse_categorical_accuracy: 0.7028 - val_loss: 0.8887 - val_sparse_categorical_accuracy: 0.6341
Epoch 4/30
7850/7850 [==============================] - 6s 744us/step - loss: 0.5605 - sparse_categorical_accuracy: 0.7650 - val_loss: 0.8103 - val_sparse_categorical_accuracy: 0.7027
Epoch 5/30
7850/7850 [==============================] - 6s 745us/step - loss: 0.5467 - spa

6297/6297 [==============================] - 5s 851us/step - loss: 0.4025 - sparse_categorical_accuracy: 0.8491 - val_loss: 4.8364 - val_sparse_categorical_accuracy: 0.3500
Epoch 4/20
6297/6297 [==============================] - 5s 848us/step - loss: 0.4253 - sparse_categorical_accuracy: 0.8418 - val_loss: 4.4396 - val_sparse_categorical_accuracy: 0.3162
Epoch 5/20
6297/6297 [==============================] - 7s 1ms/step - loss: 0.3495 - sparse_categorical_accuracy: 0.8722 - val_loss: 4.7724 - val_sparse_categorical_accuracy: 0.3621
Epoch 6/20
6297/6297 [==============================] - 7s 1ms/step - loss: 0.3518 - sparse_categorical_accuracy: 0.8745 - val_loss: 4.8621 - val_sparse_categorical_accuracy: 0.3543
Epoch 7/20
6297/6297 [==============================] - 5s 859us/step - loss: 0.3712 - sparse_categorical_accuracy: 0.8655 - val_loss: 4.8322 - val_sparse_categorical_accuracy: 0.3784
Epoch 8/20
6297/6297 [==============================] - 5s 850us/step - loss: 0.3424 - sparse_c

In [32]:
class_dict_D2 = {1,3,5,7,9,11,13,15}
class_dict_D1 = makeClassDictD1(class_dict_D2, num_classes)
print("==== Adding Half More Classes ====")
print("class dict D1: ", class_dict_D1)
print("class dict D2: ", class_dict_D2)
X_train_D1, y_train_D1, X_train_D2, y_train_D2, X_test_D1, y_test_D1, X_test_D2, y_test_D2 = makeDataD2HalfClasses(class_dict_D1, class_dict_D2, X_train, y_train, X_test, y_test)
model = vanilaLSTMs()
model.fit(X_train_D1, y_train_D1, validation_data=(X_test_D1, y_test_D1),
                    epochs=30,batch_size=64)
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=1,batch_size=64)
print("Performances after Epoch: 1")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=9,batch_size=64)
print("Performances after Epoch: 10")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))
model.fit(X_train_D2, y_train_D2, validation_data=(X_test, y_test),
                    epochs=20,batch_size=64)
print("Performances after Epoch: 30")
print("acc_D1: %f" %(accuracy_score(y_test_D1, np.argmax(model.predict(X_test_D1), axis=1))))
print("acc_D2: %f" %(accuracy_score(y_test_D2, np.argmax(model.predict(X_test_D2), axis=1))))


==== Adding Half More Classes ====
class dict D1:  {0, 2, 4, 6, 8, 10, 12, 14, 16}
class dict D2:  {1, 3, 5, 7, 9, 11, 13, 15}
Train on 6297 samples, validate on 796 samples
Epoch 1/30
6297/6297 [==============================] - 7s 1ms/step - loss: 1.8309 - sparse_categorical_accuracy: 0.2868 - val_loss: 1.4458 - val_sparse_categorical_accuracy: 0.5101
Epoch 2/30
6297/6297 [==============================] - 5s 826us/step - loss: 1.4024 - sparse_categorical_accuracy: 0.4359 - val_loss: 1.3533 - val_sparse_categorical_accuracy: 0.5025
Epoch 3/30
6297/6297 [==============================] - 5s 830us/step - loss: 1.1791 - sparse_categorical_accuracy: 0.5206 - val_loss: 1.4285 - val_sparse_categorical_accuracy: 0.4636
Epoch 4/30
6297/6297 [==============================] - 5s 835us/step - loss: 1.0040 - sparse_categorical_accuracy: 0.5865 - val_loss: 1.2096 - val_sparse_categorical_accuracy: 0.5214
Epoch 5/30
6297/6297 [==============================] - 6s 1ms/step - loss: 0.9053 - sparse_

7850/7850 [==============================] - 7s 843us/step - loss: 0.2730 - sparse_categorical_accuracy: 0.8964 - val_loss: 3.9625 - val_sparse_categorical_accuracy: 0.4448
Epoch 4/20
7850/7850 [==============================] - 8s 1ms/step - loss: 0.2451 - sparse_categorical_accuracy: 0.9089 - val_loss: 3.9045 - val_sparse_categorical_accuracy: 0.4406
Epoch 5/20
7850/7850 [==============================] - 9s 1ms/step - loss: 0.2662 - sparse_categorical_accuracy: 0.9025 - val_loss: 3.9545 - val_sparse_categorical_accuracy: 0.4400
Epoch 6/20
7850/7850 [==============================] - 7s 847us/step - loss: 0.2216 - sparse_categorical_accuracy: 0.9158 - val_loss: 4.0333 - val_sparse_categorical_accuracy: 0.4381
Epoch 7/20
7850/7850 [==============================] - 7s 836us/step - loss: 0.2398 - sparse_categorical_accuracy: 0.9104 - val_loss: 4.3036 - val_sparse_categorical_accuracy: 0.4550
Epoch 8/20
7850/7850 [==============================] - 7s 850us/step - loss: 0.2281 - sparse_c

In [67]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin), axis=1) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)), axis=1)
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)), axis=1)

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return K.mean((beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon()))


In [88]:
def dataPrepare(rand, df_l):
	train_ratio = 0.9
	X_temp, y_res = df_l[0], df_l[1]
	cut = int(X_temp.shape[0] * train_ratio)
    
	#labelencoder_y_1 = LabelEncoder()
	#y_res = labelencoder_y_1.fit_transform(y_res)
    
	train_y = y_res[:cut]
	test_y = y_res[cut:]
	train_X = X_temp[:cut]
	test_X = X_temp[cut:]

	# train_X = StandardScaler().fit_transform(train_X)
	# test_X = StandardScaler().fit_transform(test_X)

	train_X = np.reshape(train_X, (len(train_X),timesteps,-1) )
	test_X = np.reshape(test_X, (len(test_X),timesteps,-1) )

	return train_X, train_y, test_X, test_y

def runKerasModels(data_name, num_stack, hidden_layer_size, df_l):
	auc_l = []
	hist_acc = []
	hist_prec = []
	hist_rec = []
	hist_f1 = []
	path_name = 'data/mlresultsV_sanity/'
	for rand in range(1):
		best_model_name = path_name+data_name+'_'+str(rand)+'_best_model_'+str(timesteps)+'0_'+str(num_stack)+'_'+str(hidden_layer_size)+'.h5'
		train_X, train_y, test_X, test_y = dataPrepare(rand, df_l)
		print(train_X.shape, train_y.shape)
        # X = [input_size, timesteps, data_dim]
        # y = [input_size, num_classes]
		if num_stack == 1:
			# expected input data shape: (batch_size, timesteps, data_dim)
			model = Sequential()
			model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax', init='glorot_normal'))
		elif num_stack == 2:
			model = Sequential()
			model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
		elif num_stack == 4:
			model = Sequential()
			model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,return_sequences=True,init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,return_sequences=True,init='glorot_normal'))  # returns a sequence of vectors of dimension 32
			model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
			model.add(Dropout(0.2))
			model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
		else:
			print("specify the number of stacks")
		# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# 		model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=[fbeta])
		model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
		# simple early stopping
		es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=60)
		mc = ModelCheckpoint(best_model_name, monitor='val_acc', mode='max', verbose=1, save_best_only=True)
		history = model.fit(train_X, train_y, validation_data=(test_X, test_y),epochs=10,batch_size=64,verbose=0, callbacks=[es, mc])
		saved_model = load_model(best_model_name)
		test_pred = np.argmax(saved_model.predict(test_X), axis=1)
		acc = accuracy_score(test_y, test_pred)
		prfs = precision_recall_fscore_support(test_y, test_pred)
		print("Epoch ", t, "acc / prec / recall / f1 / supp : ", acc," ", prfs)
		hist_acc.append(acc)
		hist_prec.append(prfs[0])
		hist_rec.append(prfs[1])
		hist_f1.append(prfs[2])        
# 		auc_score = roc_auc_score(test_y, saved_model.predict(test_X, verbose=0))
# 		auc_l.append(auc_score)
# 		print('AUC score: %f' %(auc_score))
	return hist_acc, hist_prec, hist_rec, hist_f1

In [43]:
X_test.shape

(9894, 24, 113)

In [89]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps_l = [24]
num_stack_l = [2]
hidden_layer_size_l = [64]
for timesteps in timesteps_l:
    for num_stack in num_stack_l:
        for hidden_layer_size in hidden_layer_size_l:
            # auc_l = runKerasModels('yelp',num_stack, hidden_layer_size, [X_test, y_test])
            hist_l = runKerasModels('yelp',num_stack, hidden_layer_size, [X_train, y_train])
            print("final result: # stacks: %d, hidden_layer_size: %d, mean acc: %f, prec: %f, rec: %f, f1: %f" %(num_stack,hidden_layer_size,np.mean(hist_l[0]),np.mean(hist_l[1]),np.mean(hist_l[2]),np.mean(hist_l[3])))
#             print(hist_l)

(12732, 24, 113) (12732,)
Epoch  0 acc / prec / recall / f1 / supp :  0.059363957597173146   (array([0.05936396, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), array([0.11207472, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ]), array([ 84,  96,  58,  64, 165, 147,  53,  62,  37,  30,  31,  29,  47,
        62,  47, 363,  40]))
final result: # stacks: 2, hidden_layer_size: 64, mean acc: 0.059364, prec: 0.003492, rec: 0.058824, f1: 0.006593


In [ ]:
plt.plot(hist_acc, label="testing acc")
plt.legend()
plt.show()

plt.plot(hist_p, label="testing prec")
plt.legend()
plt.show()

plt.plot(hist_r, label="testing recall")
plt.legend()
plt.show()

plt.plot(hist_f, label="testing f1")
plt.legend()
plt.show()

In [92]:
data_dim = 113 # (# features)
num_classes = 17

# params
timesteps = 24
num_stack = 1
hidden_layer_size = 128

# train_X, train_y, test_X, test_y = dataPrepare(1, [X_train_D1, y_train_D1])

model = Sequential()
model.add(LSTM(hidden_layer_size,input_shape=(timesteps, data_dim),init='glorot_normal'))  # return a single vector of dimension 32
# model.add(LSTM(hidden_layer_size,return_sequences=True,input_shape=(timesteps, data_dim),init='glorot_normal'))  # returns a sequence of vectors of dimension 32
# model.add(LSTM(hidden_layer_size,init='glorot_normal'))  # return a single vector of dimension 32
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax',init='glorot_normal'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=tf.contrib.metrics.f1_score())



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



TypeError: f1_score() missing 2 required positional arguments: 'labels' and 'predictions'

## Pytorch implementation .... 

In [34]:
import torch
import torch.nn as nn

#!/usr/bin/env python3
from argparse import ArgumentParser
import numpy as np
import torch
from data import get_dataset, DATASET_CONFIGS
from train import train
from model import MLP
import utils

#####################
# Set parameters
#####################

# Data params
noise_var = 0
num_datapoints = 100
test_size = 0.2
num_train = int((1-test_size) * num_datapoints)

# Network params
input_size = 20
# If `per_element` is True, then LSTM reads in one timestep at a time.
per_element = True
if per_element:
    lstm_input_size = 1
else:
    lstm_input_size = input_size
# size of hidden layers
h1 = 32
output_dim = 1
num_layers = 2
learning_rate = 1e-3
num_epochs = 20
dtype = torch.float

In [36]:
#####################
# Build model
#####################

# Here we define our model as a class
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

model = LSTM(data_dim, hidden_layer_size, batch_size=64, output_dim=17, num_layers=1)

loss_fn = torch.nn.CrossEntropyLoss()

optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

LSTM(
  (lstm): LSTM(113, 128)
  (linear): Linear(in_features=128, out_features=17, bias=True)
)

In [38]:
X_train.shape

(14147, 24, 113)

In [37]:
#####################
# Train model
#####################
from sklearn.metrics import *
hist = np.zeros(num_epochs)
hist_acc = np.zeros(num_epochs)
hist_p = np.zeros(num_epochs)
hist_r = np.zeros(num_epochs)
hist_f = np.zeros(num_epochs)



num_train = int(X_test.shape[0]*0.8)
X_train_ = X_test[:num_train,:,:]
y_train_ = y_test[:num_train]

X_test_ = X_test[num_train:,:,:]
y_test_ = y_test[num_train:]

num_epoches = 30
for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    # model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()
    
    if t % 1 == 0:
        print("Epoch ", t, "CrossEntropyLoss: ", loss.item())
        y_pred = model(X_test)
        acc = accuracy_score(y_test, y_pred)
        prfs = precision_recall_fscore_support(y_train, y_pred)
        print("Epoch ", t, "acc / prec / recall / f1 / supp : ", acc," ", prfs)
        hist_acc[t] = acc
        hist_p[t] = prfs[0]
        hist_r[t] = prfs[1]
        hist_f[t] = prfs[2]
    hist[t] = loss.item()

TypeError: view() takes at most 2 arguments (3 given)

In [ ]:
#####################
# Plot preds and performance
#####################

plt.plot(y_pred.detach().numpy(), label="Preds")
plt.plot(y_train.detach().numpy(), label="Data")
plt.legend()
plt.show()

plt.plot(hist, label="Training loss")
plt.legend()
plt.show()

plt.plot(hist_acc, label="testing acc")
plt.legend()
plt.show()

plt.plot(hist_p, label="testing prec")
plt.legend()
plt.show()

plt.plot(hist_r, label="testing recall")
plt.legend()
plt.show()

plt.plot(hist_f, label="testing f1")
plt.legend()
plt.show()